In [19]:
from langchain.schema import HumanMessage, SystemMessage
import boto3
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from opensearchpy import OpenSearch, RequestsHttpConnection
import os
import boto3
import json
import sys
from langchain.schema import BaseRetriever, Document
from typing import List, Optional, Dict, Tuple
from langchain.prompts.prompt import PromptTemplate
import uuid
from datetime import datetime
from botocore.exceptions import ClientError
from operator import itemgetter
from langchain_core.runnables import RunnableLambda
from boto3.dynamodb.conditions import Key
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.schema import BaseMessage, HumanMessage, AIMessage
from langchain.schema import BaseChatMessageHistory
from langchain_aws import ChatBedrock

from langchain_core.runnables.history import RunnableWithMessageHistory
import time
from botocore.exceptions import ClientError

In [20]:
def insert_chat_message(session_id, message_content, role, timestamp=None):
    """
    Insert a chat message into the DynamoDB table.
    
    Args:
        session_id (str): The session identifier
        message_content (str): The content of the message
        role (str): The role of the message sender (e.g., 'human', 'ai')
        timestamp (str, optional): Message timestamp. Defaults to current time if None.
    
    Returns:
        dict: The response from DynamoDB
    """
    # Initialize DynamoDB client
    dynamodb = boto3.resource('dynamodb', region_name='us-west-2')
    table = dynamodb.Table('conversations-table')
    
    # Use current time if timestamp not provided
    if timestamp is None:
        sequence = int(time.time() * 1000)  # millisecond timestamp
        timestamp = datetime.utcnow().isoformat()
    else:
        # Convert timestamp to sequence number
        dt = datetime.fromisoformat(timestamp)
        sequence = int(dt.timestamp() * 1000)
    
    # Prepare item to insert
    item = {
        'session_id': session_id,
        'sequence': sequence,
        'role': role,
        'message': message_content,
        'timestamp': timestamp
    }
    
    try:
        # Insert the item into the table
        response = table.put_item(Item=item)
        print(f"Message inserted successfully: {sequence}")
        return response
    except ClientError as e:
        print(f"Error inserting message: {e.response['Error']['Message']}")
        raise

In [21]:
def get_chat_messages(session_id, ascending=True, limit=None):
    """
    Retrieve chat messages for a session, sorted by sequence.
    
    Args:
        session_id (str): The session identifier
        ascending (bool): Sort order - True for ascending (oldest first), False for descending (newest first)
        limit (int, optional): Maximum number of messages to retrieve
    
    Returns:
        list: List of message items
    """
    # Initialize DynamoDB client
    dynamodb = boto3.resource('dynamodb', region_name='us-west-2')
    table = dynamodb.Table('conversations-table')
    
    messages = []
    
    try:
        # Prepare query parameters
        query_params = {
            'KeyConditionExpression': Key('session_id').eq(session_id),
            'ScanIndexForward': ascending  # True for ascending, False for descending
        }
        
        # Add limit if specified
        if limit:
            query_params['Limit'] = limit
        
        # Query messages for the session, sorted by sequence
        response = table.query(**query_params)
        
        # Add results to our messages list
        messages.extend(response['Items'])
        
        # Handle pagination if there are more results and no limit specified
        while 'LastEvaluatedKey' in response and not limit:
            response = table.query(
                KeyConditionExpression=Key('session_id').eq(session_id),
                ScanIndexForward=ascending,
                ExclusiveStartKey=response['LastEvaluatedKey']
            )
            messages.extend(response['Items'])
        
        print(f"Found {len(messages)} messages for session {session_id}")
        return messages
        
    except ClientError as e:
        print(f"Error retrieving messages: {e.response['Error']['Message']}")
        return []

In [22]:
def delete_all_session_messages(session_id):
    """
    Delete all messages for a specific session.
    
    Args:
        session_id (str): The session identifier
    
    Returns:
        dict: Summary of deletion operation
    """
    # Initialize DynamoDB client
    dynamodb = boto3.resource('dynamodb', region_name='us-west-2')
    table = dynamodb.Table('conversations-table')
    
    try:
        # First, query to get all messages for the session
        response = table.query(
            KeyConditionExpression=Key('session_id').eq(session_id)
        )
        
        items = response['Items']
        
        # Handle pagination if there are more results
        while 'LastEvaluatedKey' in response:
            response = table.query(
                KeyConditionExpression=Key('session_id').eq(session_id),
                ExclusiveStartKey=response['LastEvaluatedKey']
            )
            items.extend(response['Items'])
        
        # If no items found, return early
        if not items:
            print(f"No messages found for session {session_id}")
            return {"deleted_count": 0}
        
        # Delete each item
        deleted_count = 0
        for item in items:
            table.delete_item(
                Key={
                    'session_id': session_id,
                    'sequence': item['sequence']
                }
            )
            deleted_count += 1
        
        print(f"Deleted {deleted_count} messages for session {session_id}")
        return {"deleted_count": deleted_count}
        
    except ClientError as e:
        print(f"Error deleting messages: {e.response['Error']['Message']}")
        raise

In [23]:
class DynamoDBHistory(BaseChatMessageHistory):
    def __init__(self, session_id: str, table_name: str = "conversations-table"):
        self.session_id = session_id
        self.table_name = table_name
        self.dynamodb = boto3.resource('dynamodb')
        self.table = self.dynamodb.Table(table_name)
    
    @property
    def messages(self) -> list[BaseMessage]:
        """DynamoDB에서 메시지 로드"""
        try:
            response = self.table.query(
                KeyConditionExpression=Key('session_id').eq(self.session_id),
                ScanIndexForward=True,  
                Limit=10,
            )
            
            messages = []
            for item in response['Items']:
                if item['role'] == 'human':
                    messages.append(HumanMessage(content=item['message']))
                elif item['role'] == 'ai':
                    messages.append(AIMessage(content=item['message']))
            
            return messages
        except Exception as e:
            print(f"메시지 로드 실패: {e}")
            return []
    
    def add_message(self, message: BaseMessage) -> None:
        """새 메시지를 DynamoDB에 저장"""
        sequence = int(time.time() * 1000)  # millisecond timestamp
        
        if isinstance(message, HumanMessage):
            role = 'human'
        elif isinstance(message, AIMessage):
            role = 'ai'
        else:
            return
        
        self.table.put_item(
            Item={
                'session_id': self.session_id,
                'sequence': sequence,
                'role': role,
                'message': message.content,
                'timestamp': datetime.now().isoformat()
            }
        )
    
    def clear(self) -> None:
        """대화 기록 삭제"""
        try:
            # Query all items for this session
            response = self.table.query(
                KeyConditionExpression=Key('session_id').eq(self.session_id)
            )
            
            # Delete each item
            for item in response['Items']:
                self.table.delete_item(
                    Key={
                        'session_id': item['session_id'],
                        'sequence': item['sequence']
                    }
                )
        except Exception as e:
            print(f"메시지 삭제 실패: {e}")

In [24]:
response1 = insert_chat_message(
    session_id="test-session-001",
    message_content="안녕하세요, 도움이 필요합니다.",
    role="human"
)

Message inserted successfully: 1762615047603


In [25]:
result=get_chat_messages(session_id="test-session-001")

Found 7 messages for session test-session-001


In [26]:
result

[{'message': '안녕하세요',
  'role': 'human',
  'session_id': 'test-session-001',
  'timestamp': '2025-11-08T15:16:38.273198',
  'sequence': Decimal('1762614998273')},
 {'message': '안녕하세요! 반갑습니다! 😊\n\n한국어로 인사해 주셔서 기쁘네요. 저는 Claude라고 하는 AI 어시스턴트입니다. 오늘 어떻게 지내고 계신가요? \n\n한국어와 영어 모두 편하게 사용하실 수 있으니, 어떤 언어로든 대화하시면 됩니다. 궁금한 것이 있으시거나 도움이 필요한 일이 있으시면 언제든 말씀해 주세요. 함께 즐거운 대화를 나눠보아요!',
  'role': 'ai',
  'session_id': 'test-session-001',
  'timestamp': '2025-11-08T15:16:38.334006',
  'sequence': Decimal('1762614998333')},
 {'message': '상사병은 무엇인가요?',
  'role': 'human',
  'session_id': 'test-session-001',
  'timestamp': '2025-11-08T15:16:54.652771',
  'sequence': Decimal('1762615014652')},
 {'message': '안녕하세요! 상사병에 대해 설명해드릴게요. 😊\n\n상사병(相思病)은 한국어에서 사용되는 표현으로, 문자 그대로는 "서로 그리워하는 병"이라는 뜻입니다. \n\n**상사병의 의미:**\n- 한 사람을 깊이 사랑하거나 그리워하여 생기는 심리적, 신체적 증상들을 비유적으로 "병"이라고 표현한 것입니다\n- 주로 짝사랑이나 이루어질 수 없는 사랑으로 인한 고통을 의미해요\n\n**일반적인 증상들:**\n- 식욕 부진이나 잠을 제대로 못 자는 것\n- 집중력 저하\n- 우울감이나 무기력감\n- 그 사람 생각에만 빠져있게 되는 상태\n- 때로는 실제로 

In [27]:
delete_all_session_messages("test-session-001")

Deleted 7 messages for session test-session-001


{'deleted_count': 7}

In [28]:
class AdvancedLLM:
    """고급 LLM 스트리밍 관리자"""
    
    def __init__(self, region_name: str = "us-west-2"):
        self.region_name = region_name
        self.model_id = None
        self.bedrock_client = self._setup_bedrock_client()
        self.llm = self._setup_llm()
    
    def _setup_bedrock_client(self):
        """Bedrock 클라이언트 설정"""
        try:
            return boto3.client(
                service_name='bedrock-runtime',
                region_name=self.region_name
            )
        except Exception as e:
            print(f"❌ Bedrock 클라이언트 초기화 실패: {e}")
            return None
    
    def _setup_llm(self, model_id="global.anthropic.claude-sonnet-4-20250514-v1:0"):
        """LLM 설정"""
        try:
            if not self.bedrock_client:
                print("❌ Bedrock 클라이언트가 없습니다.")
                return None
            self.model_id = model_id
            return ChatBedrock(
                client=self.bedrock_client,
                model_id=self.model_id,
                model_kwargs={
                    "max_tokens": 2000,
                    "temperature": 0.15,
                    "top_p": 0.9,
                }
            )
        except Exception as e:
            print(f"❌ LLM 초기화 실패: {e}")
            return None

In [29]:
llm = AdvancedLLM().llm

In [30]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory

# Your prompt template (already correct)
template_dynamo = """The following is a friendly conversation between a human and an AI. 
The AI is talkative and provides lots of specific details from its context. 

Relevant Information:

{chat_history}

Conversation:
Human: {question}
AI:"""

prompt_dynamo = PromptTemplate(
    input_variables=["chat_history", "question"], 
    template=template_dynamo
)

# Create chain with DynamoDB history
chain = prompt_dynamo | llm | StrOutputParser()




In [31]:
chain_dynamo = RunnableWithMessageHistory(
    chain,
    get_session_history=lambda session_id: DynamoDBHistory(session_id, "conversations-table"),
    input_messages_key="question",
    history_messages_key="chat_history"
)

# Usage
response = chain_dynamo.invoke(
    {"question": "안녕하세요"},
    config={"configurable": {"session_id": "test-session-001"}}
)



In [32]:
response

'안녕하세요! 반갑습니다! 😊\n\n한국어로 인사해 주셔서 기쁘네요. 저는 Claude라고 하는 AI 어시스턴트입니다. 오늘 어떻게 지내고 계신가요? \n\n한국어와 영어 모두 편하게 사용하실 수 있으니, 어떤 언어로든 대화하시면 됩니다. 궁금한 것이 있으시거나 도움이 필요한 일이 있으시면 언제든 말씀해 주세요. 함께 즐거운 대화를 나눠보아요!'

In [33]:
res1=chain_dynamo.invoke( {"question": "상사병은 무엇인가요?"},
    config={"configurable": {"session_id": "test-session-001"}})

In [34]:
res1

'안녕하세요! 상사병에 대해 설명해드릴게요! 😊\n\n상사병(相思病)은 한국어로 "짝사랑의 병" 또는 "그리움의 병"이라는 뜻입니다. 의학적인 질병은 아니고, 사랑하는 사람을 그리워하거나 짝사랑으로 인해 생기는 심리적, 신체적 증상들을 비유적으로 표현한 말이에요.\n\n**상사병의 주요 증상들:**\n- 식욕 부진이나 잠을 제대로 못 자는 것\n- 집중력 저하\n- 우울하거나 멍한 상태\n- 그 사람 생각만 계속 나는 것\n- 가슴이 답답하거나 한숨이 자주 나오는 것\n- 체중 감소나 무기력감\n\n**문화적 배경:**\n- 동양 문화권에서 오래전부터 사용되어온 표현이에요\n- 문학작품이나 드라마에서도 자주 등장하는 소재죠\n- "사랑의 아픔"을 질병에 비유한 시적인 표현이라고 할 수 있어요\n\n현대적으로는 이런 증상이 심할 경우 전문가의 도움을 받는 것이 좋다고 여겨집니다. 혹시 이런 경험이 있으시거나 궁금한 점이 더 있으시면 언제든 말씀해 주세요! 💝'

In [35]:
res2=chain_dynamo.invoke( {"question": "나는 상사병이 상사로 인해 스트레스 받아서 생기는 마음의 병이라고 나는생각해"},
    config={"configurable": {"session_id": "test-session-001"}})

In [36]:
res2

'아하! 정말 재미있는 해석이네요! 😄\n\n말씀하신 것처럼 "상사병"을 "상사(上司) + 병"으로 이해하신 거군요! 직장에서 상사 때문에 받는 스트레스로 생기는 병이라고 생각하신 것 같아요. 사실 이런 해석도 충분히 이해가 되는 게:\n\n**현실적으로 상사 스트레스는 정말 심각하죠:**\n- 직장인들이 가장 많이 겪는 스트레스 중 하나\n- 불합리한 업무 지시나 갑질\n- 소통 부재나 과도한 압박\n- 이로 인한 우울증, 불안장애, 불면증 등\n\n**실제로 이런 증상들이 나타나기도 해요:**\n- 월요병이 심해지거나\n- 출근하기 싫어지거나  \n- 식욕부진, 수면장애\n- 만성피로나 두통\n\n하지만 제가 앞서 설명한 상사병(相思病)은 한자로 "서로 그리워할 상(相) + 생각할 사(思) + 병(病)"이라서 "그리움의 병, 짝사랑의 병"이라는 뜻이에요! \n\n두 가지 모두 마음의 병이라는 점에서는 비슷하네요! 😊 어떤 의미로 궁금해하셨던 건가요?'

In [37]:
res3=chain_dynamo.invoke( {"question": "내가 생각하는 상사병은 무엇인가요?"},
    config={"configurable": {"session_id": "test-session-001"}})

In [38]:
res3

'아! 당신이 생각하시는 상사병은 **상사(上司)로 인해 스트레스를 받아서 생기는 마음의 병**이라고 하셨죠! 😊\n\n방금 전 대화에서 말씀해주신 내용을 보면:\n\n**당신이 정의한 상사병:**\n- 상사(上司) = 직장 상사, 윗사람\n- 그 상사로 인해 받는 스트레스\n- 그것이 원인이 되어 생기는 마음의 병\n\n정말 현실적이고 공감되는 해석이에요! 실제로 많은 직장인들이 겪고 있는 문제이기도 하고요. \n\n전통적인 상사병(相思病 - 짝사랑의 병)과는 완전히 다른 의미로 이해하신 거네요. 한국어의 동음이의어 특성 때문에 이런 재미있는 해석이 가능한 것 같아요!\n\n두 가지 모두 "마음의 병"이라는 공통점은 있지만:\n- 전통적 상사병 → 사랑으로 인한 그리움의 아픔\n- 당신이 생각한 상사병 → 직장 상사로 인한 스트레스성 질환\n\n어떤 계기로 이렇게 생각하게 되셨나요? 혹시 직장 경험이 있으셔서 그런 건가요? 😄'

In [39]:
delete_all_session_messages("test-session-001")

Deleted 8 messages for session test-session-001


{'deleted_count': 8}